<a href="https://colab.research.google.com/github/HanlynnKe/nlp-demo/blob/master/chinese_text_binary_classification_with_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Base code borrowed from [this Google Colab Notebook](https://colab.research.google.com/github/google-research/bert/blob/master/predicting_movie_reviews_with_bert_on_tf_hub.ipynb).

Refactored by [Hanlynn Ke](https://github.com/HanlynnKe)

Credit to [Shuyi Wang](https://www.linkedin.com/in/shuyi-wang-b3955026/)

Please refer to [this GitHub repository](https://github.com/wshuyi/demo-chinese-text-binary-classification-with-bert) for the tutorial on how to do binary classification on Chinese text data.



In [1]:
!pip install bert-tensorflow
!wget https://raw.githubusercontent.com/soaringk/BDCI-text-classification/master/data/train.csv
!wget https://raw.githubusercontent.com/soaringk/BDCI-text-classification/master/data/test_new.csv

     |████████████████████████████████| 71kB 3.4MB/s 
--2019-11-24 03:45:08--  https://raw.githubusercontent.com/soaringk/BDCI-text-classification/master/data/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1028172 (1004K) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>]   1004K  --.-KB/s    in 0.04s   

2019-11-24 03:45:09 (22.3 MB/s) - ‘train.csv’ saved [1028172/1028172]

--2019-11-24 03:45:10--  https://raw.githubusercontent.com/soaringk/BDCI-text-classification/master/data/test_new.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request 

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_hub as hub
import pickle
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

tf.enable_eager_execution()

In [0]:
def pretty_print(result):
    df = pd.DataFrame([result]).T
    df.columns = ["values"]
    return df

In [0]:
def create_tokenizer_from_hub_module(bert_model_hub):
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(bert_model_hub)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

def make_features(dataset, label_list, MAX_SEQ_LENGTH, 
                  tokenizer, DATA_COLUMN, LABEL_COLUMN):
    input_example = dataset.apply(lambda x: bert.run_classifier. InputExample(guid=None,
                                                   text_a = x[DATA_COLUMN],
                                                   text_b = None,
                                                   label = x[LABEL_COLUMN]),
                                                   axis = 1)
    features = bert.run_classifier.convert_examples_to_features(input_example, label_list, MAX_SEQ_LENGTH, tokenizer)
    return features

def create_model(bert_model_hub, is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      bert_model_hub,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)

    # regularization
    regularizer = tf.nn.l2_loss(output_weights)
    loss = tf.reduce_mean(loss + 0.01 * regularizer)

    return (loss, predicted_labels, log_probs)

# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(bert_model_hub, num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        bert_model_hub, is_predicting, input_ids, input_mask, 
          segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        f1_score = tf.contrib.metrics.f1_score(
            label_ids,
            predicted_labels)
        auc = tf.metrics.auc(
            label_ids,
            predicted_labels)
        recall = tf.metrics.recall(
            label_ids,
            predicted_labels)
        precision = tf.metrics.precision(
            label_ids,
            predicted_labels) 
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        return {
            "eval_accuracy": accuracy,
            "f1_score": f1_score,
            "auc": auc,
            "precision": precision,
            "recall": recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        bert_model_hub, is_predicting, input_ids, 
          input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn

def estimator_builder(bert_model_hub, OUTPUT_DIR, SAVE_SUMMARY_STEPS, 
                      SAVE_CHECKPOINTS_STEPS, label_list, LEARNING_RATE, 
                      num_train_steps, num_warmup_steps, BATCH_SIZE):

    # Specify outpit directory and number of checkpoint steps to save
    run_config = tf.estimator.RunConfig(
        model_dir=OUTPUT_DIR,
        save_summary_steps=SAVE_SUMMARY_STEPS,
        save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

    model_fn = model_fn_builder(
      bert_model_hub = bert_model_hub,
      num_labels=len(label_list),
      learning_rate=LEARNING_RATE,
      num_train_steps=num_train_steps,
      num_warmup_steps=num_warmup_steps)

    estimator = tf.estimator.Estimator(
      model_fn=model_fn,
      config=run_config,
      params={"batch_size": BATCH_SIZE})
    return estimator, model_fn, run_config


In [0]:
def run_on_dfs(train, test, DATA_COLUMN, LABEL_COLUMN, 
               MAX_SEQ_LENGTH = 128,
               BATCH_SIZE = 32,
               LEARNING_RATE = 2e-5,
               NUM_TRAIN_EPOCHS = 3.0,
               WARMUP_PROPORTION = 0.1,
               SAVE_SUMMARY_STEPS = 100,
               SAVE_CHECKPOINTS_STEPS = 10000,
               bert_model_hub = "https://tfhub.dev/google/bert_chinese_L-12_H-768_A-12/1"):

    label_list = train[LABEL_COLUMN].unique().tolist()
    
    tokenizer = create_tokenizer_from_hub_module(bert_model_hub)

    train_features = make_features(train, label_list, MAX_SEQ_LENGTH, tokenizer, DATA_COLUMN, LABEL_COLUMN)
    test_features = make_features(test, label_list, MAX_SEQ_LENGTH, tokenizer, DATA_COLUMN, LABEL_COLUMN)

    num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
    num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

    estimator, model_fn, run_config = estimator_builder(
                                  bert_model_hub, 
                                  OUTPUT_DIR, 
                                  SAVE_SUMMARY_STEPS, 
                                  SAVE_CHECKPOINTS_STEPS, 
                                  label_list, 
                                  LEARNING_RATE, 
                                  num_train_steps, 
                                  num_warmup_steps, 
                                  BATCH_SIZE)

    train_input_fn = bert.run_classifier.input_fn_builder(
        features=train_features,
        seq_length=MAX_SEQ_LENGTH,
        is_training=True,
        drop_remainder=False)

    estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)

    test_input_fn = run_classifier.input_fn_builder(
        features=test_features,
        seq_length=MAX_SEQ_LENGTH,
        is_training=False,
        drop_remainder=False)

    result_dict = estimator.evaluate(input_fn=test_input_fn, steps=None)
    return result_dict, estimator
    

In [0]:
def getPrediction(in_sentences, estimator):
  labels = ["Negaitive", "Positive"]
  label_list = train['label'].unique().tolist()
  MAX_SEQ_LENGTH = 128
  bert_model_hub = "https://tfhub.dev/google/bert_chinese_L-12_H-768_A-12/1"
  
  tokenizer = create_tokenizer_from_hub_module(bert_model_hub)
  input_examples = [bert.run_classifier.InputExample(guid=None, 
                                                     text_a=x, 
                                                     text_b=None, 
                                                     label=0) 
                    for x in in_sentences]
  input_features = bert.run_classifier.convert_examples_to_features(input_examples, 
                                                                    label_list, 
                                                                    MAX_SEQ_LENGTH, 
                                                                    tokenizer)
  predict_input_fn = bert.run_classifier.input_fn_builder(
      features=input_features, 
      seq_length=MAX_SEQ_LENGTH, 
      is_training=False, 
      drop_remainder=False)

  predictions = estimator.predict(predict_input_fn)
  
  return [(sentence, prediction['probabilities'], labels[prediction['labels']]) for sentence, prediction in zip(in_sentences, predictions)]

In [0]:
import random
random.seed(0)
OUTPUT_DIR = 'output'

----- you just need to focus from here ------

## Get your data

In [0]:
dataset = pd.read_csv('train.csv')
train_data, test = train_test_split(dataset, test_size=0.1, random_state=0)

train_data_pos = train_data.loc[train_data['label'] == 1]
train_data_neg = train_data.loc[train_data['label'] == 0]

train_set = []
for i in range(5):
  train = train_data_neg.sample(frac=0.7)
  train = train.append(train_data_pos)
  train = train.sample(frac=1)
  train_set.append(train)

In [0]:
myparam = {
        "DATA_COLUMN": "comment",
        "LABEL_COLUMN": "label",
        "BATCH_SIZE" : 32, # 16 12
        "LEARNING_RATE": 2e-5, #6e-5 6e-4 2e-7
        "NUM_TRAIN_EPOCHS":5,
        "bert_model_hub":"https://tfhub.dev/google/bert_chinese_L-12_H-768_A-12/1"
    }

In [12]:
auc_predictions_dict = {}
target = pd.read_csv('test_new.csv')
pred_sentences = target['comment'].values.tolist()
for train in train_set:
  result, estimator = run_on_dfs(train, test, **myparam)
  auc = result['auc']

  print(pretty_print(result))

  predictions = getPrediction(pred_sentences, estimator)

  auc_predictions_dict[auc] = predictions

  !rm -rf output/

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Writing example 0 of 6706


INFO:tensorflow:Writing example 0 of 6706


INFO:tensorflow:*** Example ***


...

INFO:tensorflow:Restoring parameters from output/model.ckpt-1047


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


In [13]:
auc_predictions_dict.keys()

dict_keys([0.94481194, 0.94698423, 0.94935954, 0.95034426, 0.949555])

In [0]:
target['count'] = 0
target['label'] = None
auc_all = sum(auc_predictions_dict.keys())

for auc, predictions in auc_predictions_dict.items():
  for row in range(len(predictions)):
    if predictions[row][2] == 'Positive':
      target.loc[row, 'count'] += auc / auc_all

for row in range(len(target)):
  if target.loc[row, 'count'] >= 0.5:
    target.loc[row, 'label'] = 1
  else:
    target.loc[row, 'label'] = 0 

In [38]:
target.head(10)

,id,comment,count,label
0,0011f384-9e54-4fb4-a272-330a6cab6804,糯米团是我小时候的记忆了，吃起还是好吃，只是小时候的油条没有这么硬！油茶也还好！可以试试,0.0,0
1,00223e4f-47e1-4fc8-9657-06444a7de9a5,满满的五星好评，口味好，服务好，特别喜欢，昨天第一次买，今天就回购了，买的刨奶，店长问我加腰...,0.0,0
2,00225350-c169-435c-84cf-970068df5b12,好喝！经常会再去买来喝！就是排队的人太多了,0.0,0
3,00a3190c-90c1-44c3-b809-7a9b1314cd27,三个人订的四人餐，菜量大没吃完，问道不错。,0.0,0
4,00b3f76e-fda3-42cd-8884-25e03a5dba64,好的一如既往，真真爱上了自助炒饭自助八宝粥自助冰粉！！！喜欢所有菜和肉，两女一男吃两份两人餐...,0.0,0
5,00b6d1c7-2f3e-4124-a5ba-5f4c4342bc94,太tmd的难吃了跟狗屎一样草,0.0,0
6,00fb0c72-569a-4cf3-8273-b602564d64ba,火锅味的味道比渝李记那个火锅米线的味道好吃，更有一种老火锅的感觉，很划算，但是有韭菜叶的就好...,0.0,0
7,0136d22b-d1a5-45af-bf47-fd5f95863011,今天肯定是走着来的，饿坏宝宝了,0.0,0
8,01662368-434b-4cf9-a885-0855c884cdf1,差评。 饭是馊的！根本吃不下，再也不会点了！菜没问题，但是不可能不吃饭啊，被迫去买了方便面...,1.0,1
9,01876c3f-d9c5-4ddf-83d1-045830755138,分量足，态度没得说！原来吃了，今天看到刚上团购，又来了！经理人很帅，还送了一砸果籽,0.0,0


In [16]:
for auc, predictions in auc_predictions_dict.items():
  print(predictions[5])

('太tmd的难吃了跟狗屎一样草', array([-1.2747266e-03, -6.6656842e+00], dtype=float32), 'Negaitive')
('太tmd的难吃了跟狗屎一样草', array([-1.0496827e-03, -6.8597693e+00], dtype=float32), 'Negaitive')
('太tmd的难吃了跟狗屎一样草', array([-9.560128e-05, -9.255679e+00], dtype=float32), 'Negaitive')
('太tmd的难吃了跟狗屎一样草', array([-1.1838817e-03, -6.7395091e+00], dtype=float32), 'Negaitive')
('太tmd的难吃了跟狗屎一样草', array([-4.7362555e-04, -7.6553636e+00], dtype=float32), 'Negaitive')


In [0]:
target_o = target[['id', 'label']]
target_o.to_csv('output_BERT_ensemble.csv', index=False)
target.to_csv('output.csv',index=False)

In [0]:
from google.colab import files
files.download('output_BERT_ensemble.csv') 